## Bloom Duck Task
this documentation for the given task done by interns Sharifah Malhan and Fatimah Emad El-Den

in this task we were assigned to find a reliable API source to retrive food data then calculate their nutrients and calories
we worked with Nutritionix API to retrive data of food nutrients and calories for sepcific recipe, in this case, Vegetable chow mein

starting by defining a general function called *Make_Api_Request*
to initiate a request to the nutritionix api

In [1]:
import requests
API_HEADERS = {
    "x-app-id": "b3d09c45",
    "x-app-key": "9ebd3d652218e3e638f359f4ff854503"
}
SEARCH_URL = "https://trackapi.nutritionix.com/v2/search/instant"
NUTRIENT_URL = "https://trackapi.nutritionix.com/v2/natural/nutrients"

def make_api_request(url, payload=None, method='GET'):
    if method == 'POST':
        response = requests.post(url, json=payload, headers=API_HEADERS)
    else:
        response = requests.get(url, headers=API_HEADERS, params=payload)
        
    if response.status_code == 200:
        return response.json()
    else:
        print(f"API Request Error: {response.status_code}, Detail: {response.text}")
        return None

The next function goes through each food name in the provided list below. For each food, it makes API requests to retrieve data related to the food, including nutrient information. The function then calculates the nutrient values based on the serving size specified.
The function keeps track of the total values for various nutrients, including calories, protein, sugar, sodium, and dietary fiber,and verious vitamins by accumulating the values for each food item. It also prints the individual nutrient values for each food item along based on given serving size

In [2]:
def get_nutrient_info(food_names):
    # Nutrient attribute ID mapping based on the documentation
    attr_id_mapping = {
        203: "Protein (g)",
        204: "Total Fat (g)",
        205: "Total Carbohydrate (g)",
        208: "Calories",
        307: "Sodium (mg)",
        318: "Vitamin A (IU)",
        320: "Vitamin A (RAE)",
        324: "Vitamin D (IU)",
        401: "Vitamin C, total ascorbic acid (mg)",
        404: "Thiamin (mg)",
        418: "Vitamin B-12 (µg)",
        430: "Vitamin K (phylloquinone) (µg)",
    }

    total_calories = 0
    total_proteins = 0
    total_sodium = 0
    total_vitamins = {key: 0 for key in attr_id_mapping.values() if key not in ["Calories", "Protein (g)", "Sodium (mg)"]}

    for food_name in food_names:
        desired_weight_g = int(input(f"How much do you want of {food_name} (in grams)? \n"))
        data = make_api_request(SEARCH_URL, {'query': food_name})
        if data and data['common']:
            common_food_name = data['common'][0]['food_name']
            nutrient_data = make_api_request(NUTRIENT_URL, {'query': common_food_name}, 'POST')
            if nutrient_data and nutrient_data['foods']:
                food_info = nutrient_data['foods'][0]
                serving_weight_grams = food_info.get('serving_weight_grams', 1)
                ratio = desired_weight_g / serving_weight_grams

                # Initialize dictionary to store the extracted nutrients
                nutrients = {name: 0 for name in attr_id_mapping.values()}

                # Extract basic nutrients and update the dictionary
                nutrients["Calories"] = round(food_info.get('nf_calories', 0) * ratio, 2)
                nutrients["Protein (g)"] = round(food_info.get('nf_protein', 0) * ratio, 2)
                nutrients["Total Fat (g)"] = round(food_info.get('nf_total_fat', 0) * ratio, 2)
                nutrients["Total Carbohydrate (g)"] = round(food_info.get('nf_total_carbohydrate', 0) * ratio, 2)
                total_calories += nutrients["Calories"]
                total_proteins += nutrients["Protein (g)"]
                total_sodium += nutrients["Sodium (mg)"]

                # Extract other nutrients from full_nutrients array
                for nutrient in food_info.get('full_nutrients', []):
                    nutrient_id = nutrient['attr_id']
                    if nutrient_id in attr_id_mapping:
                        nutrient_name = attr_id_mapping[nutrient_id]
                        nutrients[nutrient_name] = round(nutrient['value'] * ratio, 2)
                        if nutrient_name not in ["Calories", "Protein (g)", "Sodium (mg)"]:
                            total_vitamins[nutrient_name] += nutrients[nutrient_name]

                # Display the nutrient information
                print(f"\n{common_food_name}: {nutrients['Calories']} kcal, {nutrients['Protein (g)']}g protein, {nutrients['Total Fat (g)']}g fat, {nutrients['Total Carbohydrate (g)']}g carbs for {desired_weight_g}g")
                for key in attr_id_mapping.values():
                    if key not in ["Calories", "Protein (g)", "Total Fat (g)", "Total Carbohydrate (g)"]:
                        print(f"{key}: {nutrients[key]}")

    print("////////")
    print(f"Total Calories for the Recipe: {total_calories} kcal")
    print(f"Total Proteins for the Recipe: {total_proteins} g")
    print(f"Total Sodium for the Recipe: {total_sodium} mg")
    for key, value in total_vitamins.items():
        print(f"Total {key} for the Recipe: {value}")

the final step was to give a list of food item to search them as queries
the *adjustment* dictonary is used to specify an ingredient weight in Grams, if not specified, the code will still work fine with random weights

In [3]:
food_items = ['dry egg noodles', 'rapeseed oil', 'onion', 'shiitake mushrooms',
         'garlic', 'baby corn', 'tomato', 'beansprouts', 'soy sauce']
get_nutrient_info(food_items)

How much do you want of dry egg noodles (in grams)? 
200

dry rice noodles: 728.0 kcal, 11.9g protein, 1.12g fat, 160.36g carbs for 200g
Sodium (mg): 364.0
Vitamin A (IU): 0.0
Vitamin A (RAE): 0.0
Vitamin D (IU): 0.0
Vitamin C, total ascorbic acid (mg): 0.0
Thiamin (mg): 0.06
Vitamin B-12 (µg): 0.0
Vitamin K (phylloquinone) (µg): 0.0
How much do you want of rapeseed oil (in grams)? 
2

rapeseed oil: 17.68 kcal, 0.0g protein, 2.0g fat, 0.0g carbs for 2g
Sodium (mg): 0.0
Vitamin A (IU): 0.0
Vitamin A (RAE): 0.0
Vitamin D (IU): 0.0
Vitamin C, total ascorbic acid (mg): 0.0
Thiamin (mg): 0.0
Vitamin B-12 (µg): 0.0
Vitamin K (phylloquinone) (µg): 1.43
How much do you want of onion (in grams)? 
4

onion: 1.76 kcal, 0.05g protein, 0.01g fat, 0.41g carbs for 4g
Sodium (mg): 0.12
Vitamin A (IU): 0.08
Vitamin A (RAE): 0.0
Vitamin D (IU): 0.0
Vitamin C, total ascorbic acid (mg): 0.21
Thiamin (mg): 0.0
Vitamin B-12 (µg): 0.0
Vitamin K (phylloquinone) (µg): 0.02
How much do you want of shiitake mush